# Attachment Protocol Example - Alice

This notebook tests the capablity to extend the basic controller to control a custom protocol developed in an aries agent. This protocol allows you to attach a file to a message.
Run this along side [Attachment Protocol Bob](http://localhost:8889/notebooks/attachment.ipynb)

In [34]:
%autoawait
import time
import asyncio

IPython autoawait is `on`, and set to use `asyncio`


####  Intialise a controller 

In [35]:
from om_aries_controller.om_aries_controller import OMAriesController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8030
WEBHOOK_BASE = ""
ADMIN_URL = "http://alice-agent:8021"

# Based on the aca-py agent you wish to control
om_controller = OMAriesController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL)




### Confirm that an active connection exists

In [37]:
response = await om_controller.connections.get_connections()
results = response['results']
print("Results : ", results)
if len(results) > 0:
    connection = response['results'][0]
    print("Connection :", connection)
    if connection['state'] == 'active':       
        connection_id = connection["connection_id"]
        print("Active Connection ID : ", connection_id)
    else:
        print("Connection is still progressing to active state, retry in a few moments")
else:
    print("You must create a connection")
    

Results :  [{'request_id': '2817d6be-68f4-4237-8a84-780bb6305758', 'routing_state': 'none', 'invitation_mode': 'once', 'initiator': 'external', 'updated_at': '2020-09-18 11:03:44.718412Z', 'invitation_key': 'EJv5a7NDdMZJ196SdmTpZtEt7VnmiFUvkpEVfdc4wnLu', 'created_at': '2020-09-18 11:03:23.938360Z', 'state': 'active', 'connection_id': 'ea905ec6-dbd0-44bb-a639-4c23d101a7a1', 'their_did': '5j6W5xo2aPPzXKKinSQc5E', 'accept': 'manual', 'their_label': 'Bob', 'my_did': 'MmdMDrUhYfHnF2yxH4Bfyb'}]
Connection : {'request_id': '2817d6be-68f4-4237-8a84-780bb6305758', 'routing_state': 'none', 'invitation_mode': 'once', 'initiator': 'external', 'updated_at': '2020-09-18 11:03:44.718412Z', 'invitation_key': 'EJv5a7NDdMZJ196SdmTpZtEt7VnmiFUvkpEVfdc4wnLu', 'created_at': '2020-09-18 11:03:23.938360Z', 'state': 'active', 'connection_id': 'ea905ec6-dbd0-44bb-a639-4c23d101a7a1', 'their_did': '5j6W5xo2aPPzXKKinSQc5E', 'accept': 'manual', 'their_label': 'Bob', 'my_did': 'MmdMDrUhYfHnF2yxH4Bfyb'}
Active Conne

### Instantiate Listener for the Attachment topic

This is emitted using PyPubSub when the controller receives a atatchment webhook from the agent. This happens everytime the agent receives an attachment.

In [38]:

def attach_handler(payload):
    connection_id = payload['connection_id']
    print("Handle Attachment", payload, connection_id)

attach_listener = {
    "handler": attach_handler,
    "topic": "attachment"
}
loop = asyncio.get_event_loop()
loop.create_task(om_controller.listen_webhooks())
om_controller.register_listeners([attach_listener], defaults=True)

### Send an attachment over DIDcomm to Bob

In this instance a text file is sent, this will be received and saved by the handler in the [Bob notebook](http://localhost:8889/notebooks/attachment.ipynb).

In [24]:
from aiohttp import FormData
data = FormData()
data.add_field('file',
               open('test_file.txt', 'rb'),
               filename='test_file.txt',
               content_type='text/plain')
response = await om_controller.protocol.send_attachment(connection_id, data)
print('Attachment- Alice-> Bob')
print(response)


Attachment- Alice-> Bob
{'thread_id': '7365cd04-9804-4c17-8e24-033831c2a97a'}


In [21]:

pip install Pillow


Note: you may need to restart the kernel to use updated packages.


In [39]:
import base64
from aiohttp import FormData
with open("openmined.jpg", "rb") as imageFile:
    str = base64.b64encode(imageFile.read())
data = str.decode("utf-8")
f = open("image.txt", "w")
f.write(data)
data = FormData()
data.add_field('file',
               open('image.txt', 'rb'),
               filename='image.txt',
               content_type='text/plain')
response = await om_controller.protocol.send_attachment(connection_id, data)
print('Attachment- Alice-> Bob')
print(response)

Attachment- Alice-> Bob
{'thread_id': '945e26e0-357d-494a-aedb-5113cbdfefb3'}



# End of Tutorial

Be sure to terminate the controller so you can run another tutorial

In [8]:
response = await om_controller.terminate()
print(response)


None
